# Introduction

This notebook is a very basic and simple introductory primer to the method of ensembling (combining) base learning models, in particular the variant of ensembling known as Stacking. In a nutshell stacking uses as a first-level (base), the predictions of a few basic classifiers and then uses another model at the second-level to predict the output from the earlier first-level predictions.

The Titanic dataset is a prime candidate for introducing this concept as many newcomers to Kaggle start out here. Furthermore even though stacking has been responsible for many a team winning Kaggle competitions there seems to be a dearth of kernels on this topic so I hope this notebook can fill somewhat of that void.

I myself am quite a newcomer to the Kaggle scene as well and the first proper ensembling/stacking script that I managed to chance upon and study was one written in the AllState Severity Claims competition by the great Faron. The material in this notebook borrows heavily from Faron's script although ported to factor in ensembles of classifiers whilst his was ensembles of regressors. Anyway please check out his script here:

[Stacking Starter][1] : by Faron 


Now onto the notebook at hand and I hope that it manages to do justice and convey the concept of ensembling in an intuitive and concise manner.  My other standalone Kaggle [script][2] which implements exactly the same ensembling steps (albeit with different parameters) discussed below gives a Public LB score of 0.808 which is good enough to get to the top 9% and runs just under 4 minutes. Therefore I am pretty sure there is a lot of room to improve and add on to that script. Anyways please feel free to leave me any comments with regards to how I can improve


  [1]: https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867
  [2]: https://www.kaggle.com/arthurtok/titanic/simple-stacking-with-xgboost-0-808

https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

In [71]:
# Load in our libraries
import pandas as pd
import numpy as np
from scipy.stats import mode
import os 
import re
import csv
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
import datetime as dt
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

# Feature Exploration, Engineering and Cleaning 

Now we will proceed much like how most kernels in general are structured, and that is to first explore the data on hand, identify possible feature engineering opportunities as well as numerically encode any categorical features.

In [2]:
def fix_num_cols(dframe, skiplist, dropcol=1):
    n = 0
    m = 0
    df_tmp = dframe.copy()
    for col in dframe.columns.values:
        if col not in skiplist:
            if isinstance(df_tmp[col].dtype,((object))):
                try:
                    df_tmp[col].fillna(df_tmp[col].median(), inplace=True)
                except:
                    if dropcol:
                        df_tmp.drop(col, inplace=True, axis=1)
                    else:
                        df_tmp[col].fillna('NUL', inplace=True)
    return df_tmp

In [3]:
km_list = ['additional_education_km', 'basketball_km', 'big_church_km', 'big_market_km', 'big_road1_km', \
           'big_road2_km', 'bulvar_ring_km', 'bus_terminal_avto_km', 'catering_km', 'cemetery_km', \
           'church_synagogue_km', 'detention_facility_km', 'exhibition_km', 'fitness_km', 'green_zone_km', \
           'hospice_morgue_km', 'ice_rink_km', 'incineration_km', 'industrial_km', 'kindergarten_km', 'kremlin_km', \
           'market_shop_km', 'metro_km_avto', 'metro_km_walk', 'mkad_km', 'mosque_km', 'museum_km',\
           'nuclear_reactor_km', 'office_km', 'oil_chemistry_km', 'park_km', 'power_transmission_line_km',\
           'preschool_km', 'public_healthcare_km', 'public_transport_station_km', 'radiation_km', 'railroad_km', \
           'railroad_station_avto_km', 'railroad_station_walk_km', 'sadovoe_km', 'school_km', 'shopping_centers_km', \
           'stadium_km', 'swim_pool_km', 'theater_km', 'thermal_power_plant_km', 'ts_km', 'ttk_km', \
           'university_km', 'water_km', 'water_treatment_km', 'workplaces_km', 'zd_vokzaly_avto_km']
len(km_list)

53

In [18]:
slist = ['full_sq','trc_sqm_5000',  'metro_min_avto', 'trc_sqm_2000'\
        , 'area_m','raion_popul', 'life_sq', 'kitch_sq', 'diff_sq', 'plus_sq', 'km_sum']
slist0 = slist[:8] + km_list
len(slist0)

61

In [14]:
slist[:8]

['full_sq',
 'trc_sqm_5000',
 'metro_min_avto',
 'trc_sqm_2000',
 'area_m',
 'raion_popul',
 'life_sq',
 'kitch_sq']

In [81]:
def make_list(grep_name, ignore_name='ignore'):
    count_list = []
    for col in df_eda.columns:
        if grep_name in col and ignore_name not in col:
            count_list.append(col)
            
    return count_list

In [82]:
cafe_sum_min_list = ['cafe_sum_500_min_price_avg', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1500_min_price_avg', \
 'cafe_sum_2000_min_price_avg', 'cafe_sum_3000_min_price_avg', 'cafe_sum_5000_min_price_avg']
cafe_sum_max_list = ['cafe_sum_500_max_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_sum_1500_max_price_avg', '\
cafe_sum_2000_max_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_sum_5000_max_price_avg']
build_count_list = ['build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', \
 'build_count_monolith','build_count_panel', 'build_count_foam', 'build_count_slag', 'build_count_mix', \
 'build_count_before_1920','build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995',\
 'build_count_after_1995']
raion_build_list = ['raion_build_count_with_material_info', 'raion_build_count_with_builddate_info']
big_church_count_list = ['big_church_count_500', 'big_church_count_1000', 'big_church_count_1500',\
                         'big_church_count_2000', 'big_church_count_3000', 'big_church_count_5000']
cafe_count_list = make_list('cafe_count')
church_count_list = make_list('church_count', 'big_church')
leisure_count_list = make_list('leisure_count')
market_count_list = make_list('market_count')
office_count_list = make_list('office_count')
sport_count_list = make_list('sport_count')
trc_count_list = make_list('trc_count')
print(len(leisure_count_list))
print(len(office_count_list))
print(len(sport_count_list))
print(len(trc_count_list))

6
6
6
6


In [86]:
print (dt.datetime.now())
skip = ['price_cat','id','timestamp','price_doc']
dir_path = os.getcwd() + '/../../../data/all/'
print (dir_path)
df = pd.read_csv(dir_path + 'train.csv')
df_macro = pd.read_csv(dir_path + 'macro.csv')

df['price_length'] = [7.5 if cat > 6e6 and cat < 1e7 else len(str(cat)) for cat in df['price_doc'] ]    
df['price_length'] = [l if l < 8 else 8 for l in df['price_length']]    
df['price_cat'] = df['price_length'].astype('category').cat.codes    
df['quarter'] = df['timestamp'].apply(lambda x: pd.Timestamp(x).quarter)

df.loc[df.price_cat == 4, 'price_cat'] = 3

df_eda = pd.merge(df, df_macro, how='inner', on='timestamp', left_on=None, right_on=None,
     left_index=False, right_index=False, sort=True,
     suffixes=('_x', '_y'), copy=True, indicator=False,
     validate=None)

null_yr = 9999.0
yr = dt.datetime.now().year
# take care of bad dates, dates too old, too far out into the future, or NaN. Set them all to 9999.0
for i in df_eda['build_year'].index:
    if df_eda.loc[i, 'build_year'] < 1700 \
            or np.isnan(df_eda.loc[i, 'build_year']) \
            or df_eda.loc[i, 'build_year'] > yr:
        df_eda.loc[i, 'build_year'] = null_yr

df_eda.drop(['price_length','id', 'timestamp'], inplace=True, axis=1)
df_eda.loc[df_eda[df_eda['state'] == 33.0].index, 'state'] = 3.0

df_eda = fix_num_cols(df_eda, skip)

df_eda['diff_sq'] = df_eda['full_sq'] - df_eda['kitch_sq']
df_eda['plus_sq'] = df_eda['full_sq'] + df_eda['life_sq'] 
df_eda['km_sum'] = df_eda[km_list].apply(lambda x: sum(x), axis=1)

df_eda['cafe_sum_min_total'] = df_eda[cafe_sum_min_list].apply(lambda x: sum(x), axis=1)
df_eda['cafe_sum_max_total'] = df_eda[cafe_sum_max_list].apply(lambda x: sum(x), axis=1)    
df_eda['build_count_total'] = df_eda[build_count_list].apply(lambda x: sum(x), axis=1)
df_eda['raion_build_count_total'] = df_eda[raion_build_list].apply(lambda x: sum(x), axis=1)
df_eda['big_church_count_total'] = df_eda[big_church_count_list].apply(lambda x: sum(x), axis=1)
df_eda['cafe_count_total'] = df_eda[cafe_count_list].apply(lambda x: sum(x), axis=1)
df_eda['church_count_total'] = df_eda[church_count_list].apply(lambda x: sum(x), axis=1)
df_eda['leisure_count_total'] = df_eda[leisure_count_list].apply(lambda x: sum(x), axis=1)
df_eda['market_count_total'] = df_eda[market_count_list].apply(lambda x: sum(x), axis=1)
df_eda['office_count_total'] = df_eda[office_count_list].apply(lambda x: sum(x), axis=1)
df_eda['sport_count_total'] = df_eda[sport_count_list].apply(lambda x: sum(x), axis=1)
df_eda['trc_count_total'] = df_eda[trc_count_list].apply(lambda x: sum(x), axis=1)

x_list = [col for col in df_eda.columns if col not in skip]

scaler = StandardScaler()
scaler.fit(df_eda)
df_eda = pd.DataFrame(scaler.transform(df_eda), columns=df_eda.columns)

df_eda['price_cat'] = df['price_cat']


print (dt.datetime.now())    

2018-11-26 18:08:40.761844
/Users/chadleonard/Springboard/work/springboard/capstone/projects/capstone1_Sberbank/../../../data/all/
2018-11-26 18:09:03.173075


In [113]:
sum(df_eda.isnull().sum())

0

In [102]:
x_list = [col for col in df_eda.columns if col not in skip]
len(x_list)

386

In [73]:
catlist = []
rest = []
with open('cols.csv', 'r') as csvfile:
    file = csv.reader(csvfile, delimiter=' ')
    for row in file:
        cl = row[0].split(",")[1].replace('"','').replace("'",'').replace('(','')
        if cl not in skip:
            tp = row[0].split(",")[0]
            if tp == 'cat':
                catlist.append(cl)
            else:
                rest.append(cl)

In [114]:
#df_eda['price_cat'] = df['price_cat']
df_train = df_eda[rest + ['price_cat']].copy()
df_train.head()

,additional_education_km,apartment_build,area_m,average_provision_of_build_contract,average_provision_of_build_contract_moscow,balance_trade,balance_trade_growth,basketball_km,baths_share,big_church_km,...,raion_build_count_total,big_church_count_total,cafe_count_total,church_count_total,leisure_count_total,market_count_total,office_count_total,sport_count_total,trc_count_total,price_cat
0,-0.249330,-5.857459,-0.544788,-1.774799,1.093412,-0.836632,-0.681054,-0.235595,6.282208,-0.570582,...,-0.424498,-0.151208,-0.159297,-0.222977,-0.357060,-0.298767,-0.235116,0.054059,1.114841,1
1,-0.167851,-5.857459,-0.390702,-1.774799,1.093412,-0.836632,-0.681054,-0.672067,6.282208,-0.456069,...,-0.293118,-0.135265,-0.201367,-0.056817,0.270678,0.462241,-0.126137,0.176379,0.426275,1
2,-0.613638,-5.857459,-0.622239,-1.774799,1.093412,-0.836632,-0.681054,-0.525425,6.282208,0.284143,...,0.042194,-0.262810,-0.247096,-0.076366,-0.267383,1.984257,-0.231813,0.164147,-0.090149,1
3,-0.285556,-5.857459,-0.245700,-1.774799,1.093412,-0.866722,-0.681054,0.025245,6.282208,-0.434657,...,0.546143,-0.326582,-0.300142,-0.467330,-0.357060,0.027380,-0.357305,-0.288435,-0.004078,3
4,-0.339201,-5.857459,-0.448374,-1.774799,1.093412,-0.866722,-0.681054,-0.830288,6.282208,-0.653286,...,1.673655,3.802687,3.497186,3.999442,3.364532,1.440680,3.869773,2.745089,3.223573,3


In [101]:
sum(df_train.isnull().sum())

0

In [115]:
X_train, X_test, y_train, y_test = train_test_split(df_train.loc[:, rest], \
                                                                df_train.loc[:, 'price_cat'], test_size=0.3)
X_train.shape

(21329, 336)

All right so now having cleaned the features and extracted relevant information and dropped the categorical columns our features should now all be numeric, a format suitable to feed into our Machine Learning models. However before we proceed let us generate some simple correlation and distribution plots of our transformed dataset to observe ho

## Visualisations 

In [5]:
X_train.head(3)

,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
505,1,1,1,0,3,1,42,1,2,0,1
321,3,1,1,0,0,0,16,0,1,1,1
783,3,1,2,2,2,0,22,0,4,0,1


**Pearson Correlation Heatmap**

let us generate some correlation plots of the features to see how related one feature is to the next. To do so, we will utilise the Seaborn plotting package which allows us to plot heatmaps very conveniently as follows

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

**Takeaway from the Plots**

One thing that that the Pearson Correlation plot can tell us is that there are not too many features strongly correlated with one another. This is good from a point of view of feeding these features into your learning model because this means that there isn't much redundant or superfluous data in our training set and we are happy that each feature carries with it some unique information. Here are two most correlated features are that of Family size and Parch (Parents and Children). I'll still leave both features in for the purposes of this exercise.

**Pairplots**

Finally let us generate some pairplots to observe the distribution of data from one feature to the other. Once again we use Seaborn to help us.

In [ ]:
g = sns.pairplot(train[[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Fare', u'Embarked',
       u'FamilySize', u'Title']], hue='Survived', palette = 'seismic',size=1.2,diag_kind = 'kde',\
                 diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])

# Ensembling & Stacking models

Finally after that brief whirlwind detour with regards to feature engineering and formatting, we finally arrive at the meat and gist of the this notebook.

Creating a Stacking ensemble!

### Helpers via Python Classes

Here we invoke the use of Python's classes to help make it more convenient for us. For any newcomers to programming, one normally hears Classes being used in conjunction with Object-Oriented Programming (OOP). In short, a class helps to extend some code/program for creating objects (variables for old-school peeps) as well as to implement functions and methods specific to that class.

In the section of code below, we essentially write a class *SklearnHelper* that allows one to extend the inbuilt methods (such as train, predict and fit) common to all the Sklearn classifiers. Therefore this cuts out redundancy as  won't need to write the same methods five times if we wanted to invoke five different classifiers.

In [73]:
print(X_train.shape)
print(X_test.shape)

(623, 11)
(268, 11)


In [116]:
# Some useful parameters which will come in handy later on
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        #print(self.clf.fit(x,y).feature_importances_)
        return self.clf.fit(x,y).feature_importances_
    
# Class to extend XGboost classifer

Bear with me for those who already know this but for people who have not created classes or objects in Python before, let me explain what the code given above does. In creating my base classifiers, I will only use the models already present in the Sklearn library and therefore only extend the class for that.

**def init** : Python standard for invoking the default constructor for the class. This means that when you want to create an object (classifier), you have to give it the parameters of clf (what sklearn classifier you want), seed (random seed) and params (parameters for the classifiers).

The rest of the code are simply methods of the class which simply call the corresponding methods already existing within the sklearn classifiers. Essentially, we have created a wrapper class to extend the various Sklearn classifiers so that this should help us reduce having to write the same code over and over when we implement multiple learners to our stacker.

### Out-of-Fold Predictions

Now as alluded to above in the introductory section, stacking uses predictions of base classifiers as input for training to a second-level model. However one cannot simply train the base models on the full training data, generate predictions on the full test set and then output these for the second-level training. This runs the risk of your base model predictions already having "seen" the test set and therefore overfitting when feeding these predictions.

In [123]:
oof_test_skf = np.empty((NFOLDS, ntest))
oof_test_skf

array([[ 0.00000000e+000,  6.00000000e+000,  0.00000000e+000, ...,
         0.00000000e+000,  2.47032823e-323,  5.00000000e+000],
       [ 1.03753786e-322,  1.18575755e-322,  1.97626258e-323, ...,
         1.30000000e+001,  2.91498731e-322,  3.06320700e-322],
       [ 2.47032823e-323,  9.48278674e-001,  1.65289256e-001, ...,
         5.33590898e-322,  1.48219694e-323,  4.29727039e+000],
       [ 4.98866213e-001,  1.03753786e-322,  2.10000000e+001, ...,
        -2.00000000e+000,  0.00000000e+000,  1.48219694e-323],
       [ 3.00000000e+000,              nan,              nan, ...,
         9.88131292e-322,  2.00000000e+002,  2.27270197e-322]])

In [117]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain))
    oof_test = np.zeros((ntest))
    oof_test_skf = np.empty((NFOLDS, ntest))
    print(len(oof_test_skf))
    print(ntest)

    for i, (train_index, test_index) in enumerate(kf):
        #looping thru the Kfolds and getting a new set of train and test indexes to train and test with
        #different data is being trained and tested for each fold. 
        # The test 
        #print(i, train_index, test_index)
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        #oof_train[test_index] = [val[0] for val in clf.predict_proba(x_te)] # test portion (x_te) of train data is predicted. used to train 
                                                  # XGB model
        oof_test_skf[i, :] = clf.predict(x_test)
        oof_test_skf[i, :] = [val[0] for val in clf.predict_proba(x_test)]  # the ORIGINAL test (x_test) data is being predicted against
        #print(oof_test_skf[i, :])                # each fold's trained data ... used for XGB prediction
                                                  # that's why it's necessary to take the mean of oof_test_skf 
    oof_test[:] = oof_test_skf.mean(axis=0)      # for every column across all 5 folds (rows)
    #oof_test[:] = oof_test_skf.median(axis=0)       # want to try the mode. 
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# Generating our Base First-Level Models 

So now let us prepare five learning models as our first level classification. These models can all be conveniently invoked via the Sklearn library and are listed as follows:

 1. Random Forest classifier
 2. Extra Trees classifier
 3. AdaBoost classifer
 4. Gradient Boosting classifer
 5. Support Vector Machine

**Parameters**

Just a quick summary of the parameters that we will be listing here for completeness,

**n_jobs** : Number of cores used for the training process. If set to -1, all cores are used.

**n_estimators** : Number of classification trees in your learning model ( set to 10 per default)

**max_depth** : Maximum depth of tree, or how much a node should be expanded. Beware if set to too high  a number would run the risk of overfitting as one would be growing the tree too deep

**verbose** : Controls whether you want to output any text during the learning process. A value of 0 suppresses all text while a value of 3 outputs the tree learning process at every iteration.

 Please check out the full description via the official Sklearn website. There you will find that there are a whole host of other useful parameters that you can play around with. 

In [41]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025,
    'probability': True
    }

Furthermore, since having mentioned about Objects and classes within the OOP framework, let us now create 5 objects that represent our 5 learning models via our Helper Sklearn Class we defined earlier.

In [118]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

**Creating NumPy arrays out of our train and test sets**

Great. Having prepared our first layer base models as such, we can now ready the training and test test data for input into our classifiers by generating NumPy arrays out of their original dataframes as follows:

In [119]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = y_train.ravel()
#train = train.drop(['Survived'], axis=1)
x_train = X_train.values # Creates an array of the train data
x_test = X_test.values # Creats an array of the test data

In [121]:
sum(X_train.isnull().sum())

0

**Output of the First level Predictions** 

We now feed the training and test data into our 5 base classifiers and use the Out-of-Fold prediction function we defined earlier to generate our first level predictions. Allow a handful of minutes for the chunk of code below to run.

In [122]:
# Create our OOF train and test predictions. These base results will be used as new features
print (dt.datetime.now())
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print (dt.datetime.now())
print("Training is complete")

2018-11-26 18:18:16.783129
5
9142
5
9142
5
9142
5
9142
5
9142
2018-11-26 21:19:18.830025
Training is complete


In [80]:
print(et_oof_train[:5])
print(rf_oof_train[:5])

[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


**Feature importances generated from the different classifiers**

Now having learned our the first-level classifiers, we can utilise a very nifty feature of the Sklearn models and that is to output the importances of the various features in the training and test sets with one very simple line of code.

As per the Sklearn documentation, most of the classifiers are built in with an attribute which returns feature importances by simply typing in **.feature_importances_**. Therefore we will invoke this very useful attribute via our function earliand plot the feature importances as such

In [81]:
rf_features = list(rf.feature_importances(x_train,y_train))
et_features = list(et.feature_importances(x_train, y_train))
ada_features = list(ada.feature_importances(x_train, y_train))
gb_features = list(gb.feature_importances(x_train,y_train))

So I have not yet figured out how to assign and store the feature importances outright. Therefore I'll print out the values from the code above and then simply copy and paste into Python lists as below (sorry for the lousy hack)

In [82]:
print(rf_features)

[0.11144839790862295, 0.2091654961377744, 0.03195387964488188, 0.0237952901948958, 0.05093681370134516, 0.026056111953403346, 0.13373803812264057, 0.06073906581953875, 0.06550237584296349, 0.01628871485029207, 0.2703758158236417]


In [ ]:
#rf_features = [0.10474135,  0.21837029,  0.04432652,  0.02249159,  0.05432591,  0.02854371
#  ,0.07570305,  0.01088129 , 0.24247496,  0.13685733 , 0.06128402]
#et_features = [ 0.12165657,  0.37098307  ,0.03129623 , 0.01591611 , 0.05525811 , 0.028157
#  ,0.04589793 , 0.02030357 , 0.17289562 , 0.04853517,  0.08910063]
#ada_features = [0.028 ,   0.008  ,      0.012   ,     0.05866667,   0.032 ,       0.008
#  ,0.04666667 ,  0.     ,      0.05733333,   0.73866667,   0.01066667]
#gb_features = [ 0.06796144 , 0.03889349 , 0.07237845 , 0.02628645 , 0.11194395,  0.04778854
#  ,0.05965792 , 0.02774745,  0.07462718,  0.4593142 ,  0.01340093]

Create a dataframe from the lists containing the feature importance data for easy plotting via the Plotly package.

In [84]:
cols = X_train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

**Interactive feature importances via Plotly scatterplots**

I'll use the interactive Plotly package at this juncture to visualise the feature importances values of the different classifiers  via a plotly scatter plot by calling "Scatter" as follows:

In [47]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

Now let us calculate the mean of all the feature importances and store it as a new column in the feature importance dataframe.

In [48]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

,features,Random Forest feature importances,Extra Trees feature importances,AdaBoost feature importances,Gradient Boost feature importances,mean
0,Pclass,0.125677,0.120946,0.030,0.067730,0.086088
1,Sex,0.201063,0.376283,0.012,0.042645,0.157998
2,Age,0.029452,0.028624,0.018,0.096078,0.043039


**Plotly Barplot of Average Feature Importances**

Having obtained the mean feature importance across all our classifiers, we can plot them into a Plotly bar plot as follows:

In [49]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

# Second-Level Predictions from the First-level Output

**First-level output as new features**

Having now obtained our first-level predictions, one can think of it as essentially building a new set of features to be used as training data for the next classifier. As per the code below, we are therefore having as our new columns the first-level predictions from our earlier classifiers and we train the next classifier on this.

In [79]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

,RandomForest,ExtraTrees,AdaBoost,GradientBoost
0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0


**Correlation Heatmap of the Second Level Training set**

In [51]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

There have been quite a few articles and Kaggle competition winner stories about the merits of having trained models that are more uncorrelated with one another producing better scores.

In [123]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

Having now concatenated and joined both the first-level train and test predictions as x_train and x_test, we can now fit a second-level learning model.

### Second level learning model via XGBoost

Here we choose the eXtremely famous library for boosted tree learning model, XGBoost. It was built to optimize large-scale boosted tree algorithms. For further information about the algorithm, check out the [official documentation][1].

  [1]: https://xgboost.readthedocs.io/en/latest/

Anyways, we call an XGBClassifier and fit it to the first-level train and target data and use the learned model to predict the test data as follows:

In [37]:
X_train.shape

(623, 11)

In [124]:
len(x_train[:5][0])

5

In [125]:
print (dt.datetime.now())    
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 10,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 #objective= 'binary:logistic',
 objective= 'multi:softmax',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)
print(gbm.predict_proba(x_test)[:5])
print(gbm.score(x_train,y_train))
print(gbm.score(x_test,y_test))
print(y_test[:5])
print(predictions[:5])
print (dt.datetime.now())    

2018-11-26 21:48:28.758712
[[2.3599971e-02 7.7287185e-01 2.0337333e-01 1.5482095e-04]
 [2.3599971e-02 7.7287185e-01 2.0337333e-01 1.5482095e-04]
 [2.3599971e-02 7.7287185e-01 2.0337333e-01 1.5482095e-04]
 [2.3599971e-02 7.7287185e-01 2.0337333e-01 1.5482095e-04]
 [2.3599971e-02 7.7287185e-01 2.0337333e-01 1.5482095e-04]]
0.8077734539828403
0.45154233209363376
7105     1
3217     3
10888    1
2505     3
26915    1
Name: price_cat, dtype: int8
[1 1 1 1 1]
2018-11-26 21:50:30.939459


In [92]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 #objective= 'binary:logistic',
 objective= 'multi:softmax',
 nthread= -1,
 scale_pos_weight=1).fit(X_train, y_train)
predictions = gbm.predict(X_test)
print(gbm.predict_proba(X_test)[:5])
print(gbm.score(X_train,y_train))
print(gbm.score(X_test,y_test))
print(y_test[:5])
print(predictions[:5])

[[0.513219   0.48678097]
 [0.67727995 0.32272005]
 [0.9845808  0.01541916]
 [0.921968   0.07803201]
 [0.58239675 0.41760322]]
0.9341894060995185
0.8134328358208955
18     0
882    0
103    0
182    0
724    1
Name: Survived, dtype: int64
[0 0 0 0 0]


Just a quick run down of the XGBoost parameters used in the model:

**max_depth** : How deep you want to grow your tree. Beware if set to too high a number might run the risk of overfitting.

**gamma** : minimum loss reduction required to make a further partition on a leaf node of the tree. The larger, the more conservative the algorithm will be.

**eta** : step size shrinkage used in each boosting step to prevent overfitting

In [91]:
print(sum(y_test == predictions) / len(y_test))

0.8097014925373134


**Producing the Submission file**

Finally having trained and fit all our first-level and second-level models, we can now output the predictions into the proper format for submission to the Titanic competition as follows:

In [ ]:
# Generate Submission File 
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
StackingSubmission.to_csv("StackingSubmission.csv", index=False)

**Steps for Further Improvement**

As a closing remark it must be noted that the steps taken above just show a very simple way of producing an ensemble stacker. You hear of ensembles created at the highest level of Kaggle competitions which involves monstrous combinations of stacked classifiers as well as levels of stacking which go to more than 2 levels. 

Some additional steps that may be taken to improve one's score could be:

 1. Implementing a good cross-validation strategy in training the models to find optimal parameter values
 2. Introduce a greater variety of base models for learning. The more uncorrelated the results, the better the final score.

### Conclusion

I have this notebook has been helpful somewhat in introducing a working script for stacking learning models. Again credit must be extended to Faron and Sina. 

For other excellent material on stacking or ensembling in general, refer to the de-facto Must read article on the website MLWave: [Kaggle Ensembling Guide][1]. 

Till next time, Peace Out

  [1]: http://mlwave.com/kaggle-ensembling-guide/